In [1]:
import numpy as np
import pandas as pd
import networkx as nx
import pickle
from sklearn.externals import joblib

# Without Batch Normalization

In [2]:
x_train = joblib.load('../datasets/xtrain18.pkl')
data_y = joblib.load('../datasets/ytrain.pkl')

In [3]:
import tensorflow as tf

In [4]:
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(x_train, data_y, test_size = 0.25, random_state =123)

In [5]:
test_x.shape

(250, 10, 100, 18)

In [6]:
def createWeight(size):
    return tf.Variable(tf.truncated_normal(size, stddev=0.1))

def createBias(size):
    return tf.Variable(tf.constant(0.1,shape=size))

In [7]:
w =10
k =10
graph_width = w*k
graph_height = k
nchannels = 18 
batch_size = 32
g = tf.Graph()
with g.as_default():
    graph_input = tf.placeholder(tf.float32,shape=(batch_size,graph_height,graph_width,nchannels))
    targets = tf.placeholder(tf.float32,shape=(batch_size,2))
    pkeep = tf.placeholder(tf.float32)

    testX = tf.constant(test_x,dtype=tf.float32)
    testy = tf.constant(test_y,dtype=tf.float32)
    
    
    W_conv1 = createWeight([k,k,nchannels,64])
    b_conv1 = createBias([64])
    
    W_conv2 = createWeight([1,10,64,32])
    b_conv2 = createBias([32])
    
    W = createWeight([1*(w-9)*32,128])
    B = createBias([128])
    
    W_fc2 = createWeight([128,2])
    b_fc2 = createBias([2])
    
    
    #model
    def model(data_input):
        Y1 = tf.nn.relu(tf.nn.conv2d(data_input, W_conv1, strides=[1, w, w, 1], padding='VALID') + b_conv1)
        #Y1_drop = tf.nn.dropout(Y1, pkeep)

        Y2 = tf.nn.relu(tf.nn.conv2d(Y1, W_conv2, strides=[1, 1, 1, 1], padding='VALID') + b_conv2)
        #Y2_drop = tf.nn.dropout(Y2, pkeep)

        Y2_flat = tf.reshape(Y2, [-1, 1*(w-9)*32])
        Y_fc1 = tf.nn.relu(tf.matmul(Y2_flat, W) + B)
        Y_drop = tf.nn.dropout(Y_fc1, pkeep)

        Y_conv = tf.matmul(Y_fc1, W_fc2) + b_fc2
        return Y_conv

    #train
    logit_train = model(graph_input)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits= logit_train,labels = targets))
    
    opt = tf.train.AdamOptimizer(learning_rate=.001).minimize(loss)
    
    train_accuracy = tf.reduce_mean(tf.cast(tf.equal(
        tf.arg_max(tf.nn.softmax(logit_train),1),tf.arg_max(targets,1)),tf.float32))
    
    #test
    logit_test = model(testX)
    test_accuracy = tf.reduce_mean(tf.cast(tf.equal(
        tf.arg_max(tf.nn.softmax(logit_test),1),tf.arg_max(testy,1)),tf.float32))
    
    init = tf.global_variables_initializer()

In [8]:
sess=tf.Session(graph=g)
sess.run(init)
losses = []

In [9]:
from tqdm import tqdm

In [10]:
num_steps = 10000
for s in tqdm(range(num_steps)):
    offset = (s*batch_size) % (train_x.shape[0]-batch_size)
    fd = {graph_input:train_x[offset:offset+batch_size,:],
         targets:train_y[offset:offset+batch_size,:]}
    l,_ = sess.run([loss,opt],feed_dict=fd)
    losses.append(l)
    if s % 1000 == 0:
        print('train acc: ',sess.run([train_accuracy],feed_dict=fd))
        print('test acc: ',sess.run([test_accuracy],feed_dict=fd))

  0%|          | 42/10000 [00:00<37:13,  4.46it/s]

('train acc: ', [0.4375])
('test acc: ', [0.47599998])


 11%|█         | 1067/10000 [00:03<00:22, 396.48it/s]

('train acc: ', [0.78125])
('test acc: ', [0.55199999])


 20%|██        | 2042/10000 [00:05<00:19, 415.90it/s]

...

 81%|████████  | 8052/10000 [00:20<00:04, 421.43it/s]

('train acc: ', [0.5])
('test acc: ', [0.5])


 90%|█████████ | 9048/10000 [00:22<00:02, 420.44it/s]

('train acc: ', [0.5])
('test acc: ', [0.5])


100%|██████████| 10000/10000 [00:24<00:00, 405.11it/s]


In [11]:
print('train acc: ',sess.run([train_accuracy],feed_dict=fd))
print('test acc: ',sess.run([test_accuracy],feed_dict=fd))

('train acc: ', [0.5625])
('test acc: ', [0.5])


## Versi 2

In [12]:
w =10
k =10
graph_width = w*k
graph_height = k
nchannels = 18 
batch_size = 32
g = tf.Graph()
with g.as_default():
    graph_input = tf.placeholder(tf.float32,shape=(batch_size,graph_height,graph_width,nchannels))
    targets = tf.placeholder(tf.float32,shape=(batch_size,2))
    pkeep = tf.placeholder(tf.float32)

    testX = tf.constant(test_x,dtype=tf.float32)
    testy = tf.constant(test_y,dtype=tf.float32)
    
    
    W_conv1 = createWeight([k,k,nchannels,16])
    b_conv1 = createBias([16])
    
    W_conv2 = createWeight([1,10,16,8])
    b_conv2 = createBias([8])
    
    W = createWeight([1*(w-9)*8,32])
    B = createBias([32])
    
    W_fc2 = createWeight([32,2])
    b_fc2 = createBias([2])
    
    
    #model
    def model(data_input):
        Y1 = tf.nn.relu(tf.nn.conv2d(data_input, W_conv1, strides=[1, w, w, 1], padding='VALID') + b_conv1)
        #Y1_drop = tf.nn.dropout(Y1, pkeep)

        Y2 = tf.nn.relu(tf.nn.conv2d(Y1, W_conv2, strides=[1, 1, 1, 1], padding='VALID') + b_conv2)
        #Y2_drop = tf.nn.dropout(Y2, pkeep)

        Y2_flat = tf.reshape(Y2, [-1, 1*(w-9)*8])
        Y_fc1 = tf.nn.relu(tf.matmul(Y2_flat, W) + B)
        Y_drop = tf.nn.dropout(Y_fc1, pkeep)

        Y_conv = tf.matmul(Y_fc1, W_fc2) + b_fc2
        return Y_conv

    #train
    logit_train = model(graph_input)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits= logit_train,labels = targets))
    
    opt = tf.train.AdamOptimizer(learning_rate=.001).minimize(loss)
    
    train_accuracy = tf.reduce_mean(tf.cast(tf.equal(
        tf.arg_max(tf.nn.softmax(logit_train),1),tf.arg_max(targets,1)),tf.float32))
    
    #test
    logit_test = model(testX)
    test_accuracy = tf.reduce_mean(tf.cast(tf.equal(
        tf.arg_max(tf.nn.softmax(logit_test),1),tf.arg_max(testy,1)),tf.float32))
    
    init = tf.global_variables_initializer()

In [13]:
sess=tf.Session(graph=g)
sess.run(init)
losses = []

In [14]:
num_steps = 10000
for s in tqdm(range(num_steps)):
    offset = (s*batch_size) % (train_x.shape[0]-batch_size)
    fd = {graph_input:train_x[offset:offset+batch_size,:],
         targets:train_y[offset:offset+batch_size,:]}
    l,_ = sess.run([loss,opt],feed_dict=fd)
    losses.append(l)
    if s % 1000 == 0:
        print('train acc: ',sess.run([train_accuracy],feed_dict=fd))
        print('test acc: ',sess.run([test_accuracy],feed_dict=fd))

  1%|          | 71/10000 [00:00<00:33, 296.14it/s]

('train acc: ', [0.40625])
('test acc: ', [0.44800004])


 11%|█         | 1068/10000 [00:02<00:19, 448.27it/s]

('train acc: ', [0.53125])
('test acc: ', [0.528])


 21%|██        | 2080/10000 [00:04<00:17, 447.94it/s]

...

 81%|████████  | 8069/10000 [00:17<00:04, 457.82it/s]

('train acc: ', [0.5])
('test acc: ', [0.5])


 91%|█████████ | 9074/10000 [00:20<00:02, 439.38it/s]

('train acc: ', [0.5])
('test acc: ', [0.5])


100%|██████████| 10000/10000 [00:22<00:00, 448.07it/s]


In [15]:
print('train acc: ',sess.run([train_accuracy],feed_dict=fd))
print('test acc: ',sess.run([test_accuracy],feed_dict=fd))

('train acc: ', [0.5625])
('test acc: ', [0.49999997])


# CNNs With Batch Normalization

In [16]:
def createWeightsBN(s):
    """
    Creates weights for batch normalization layer
    
    Parameters:
    -----------
    s: int
        size of to be normalized
    """
    gamma = tf.Variable(tf.truncated_normal([s]))
    beta = tf.Variable(tf.ones([s]))
    return [gamma,beta]

def BN(x,variables,ri=[0,1,2],eps=.0001):
    """
    Applies Batch Normalization
    
    Parameters:
    ------------
    x: tensor
        the data to normalize
        
    variables: [gamma,beta]
        parameters to learn
        
    ri (optional): [int]
        reduction indicies
    eps (optional): float
        small number to keep from dividing by zero
    """
    gamma,beta = variables[0],variables[1]
    
    mu = tf.reduce_mean(x,ri,keep_dims=True)
    sigma = tf.reduce_mean(tf.square(x-mu),ri,keep_dims=True)
    x_hat = (x-mu)/(tf.sqrt(sigma+eps))
    y = gamma*x_hat+beta
    return y

In [17]:
w =10
k =10
graph_width = w*k
graph_height = k
nchannels = 18 
batch_size = 50
g = tf.Graph()
with g.as_default():
    graph_input = tf.placeholder(tf.float32,shape=(batch_size,graph_height,graph_width,nchannels))
    targets = tf.placeholder(tf.float32,shape=(batch_size,2))
    pkeep = tf.placeholder(tf.float32)

    testX = tf.constant(test_x,dtype=tf.float32)
    testy = tf.constant(test_y,dtype=tf.float32)
    
    bn_input = createWeightsBN(nchannels)
    W_conv1 = createWeight([k,k,nchannels,12])
    b_conv1 = createBias([12])
    
    bn_1 = createWeightsBN(12)
    W_conv2 = createWeight([1,10,12,12])
    b_conv2 = createBias([12])
    
    bn_2 = createWeightsBN(12)
    W = createWeight([1*(w-9)*12,12])
    B = createBias([12])
    
    bn_3 = createWeightsBN(12)
    W_fc2 = createWeight([12,2])
    b_fc2 = createBias([2])
    bn_fc2 = createWeightsBN(2)
    
    #model
    def model(data_input):
        data_input_bn = BN(data_input,bn_input)
        Y1 = tf.nn.relu(tf.nn.conv2d(data_input_bn, W_conv1, strides=[1, w, w, 1], padding='VALID') + b_conv1)
        #Y1_drop = tf.nn.dropout(Y1, pkeep)
        
        y1_bn = BN(Y1,bn_1)
        Y2 = tf.nn.relu(tf.nn.conv2d(Y1, W_conv2, strides=[1, 1, 1, 1], padding='VALID') + b_conv2)
        #Y2_drop = tf.nn.dropout(Y2, pkeep)
        
        y2_bn = BN(Y2,bn_2)
        Y2_flat = tf.reshape(Y2, [-1, 1*(w-9)*12])
        Y_fc1 = tf.nn.relu(tf.matmul(Y2_flat, W) + B)
        Y_drop = tf.nn.dropout(Y_fc1, pkeep)

        Y_conv = tf.matmul(Y_fc1, W_fc2) + b_fc2
        Y_conv_bn = BN(Y_conv,bn_fc2,ri=[0])
        return Y_conv_bn

    #train
    logit_train = model(graph_input)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits= logit_train,labels = targets))
    
    opt = tf.train.RMSPropOptimizer(learning_rate=0.00001).minimize(loss)
    
    train_accuracy = tf.reduce_mean(tf.cast(tf.equal(
        tf.arg_max(tf.nn.softmax(logit_train),1),tf.arg_max(targets,1)),tf.float32))
    
    #test
    logit_test = model(testX)
    test_accuracy = tf.reduce_mean(tf.cast(tf.equal(
        tf.arg_max(tf.nn.softmax(logit_test),1),tf.arg_max(testy,1)),tf.float32))
    
    init = tf.global_variables_initializer()

In [18]:
sess=tf.Session(graph=g)
sess.run(init)
losses = []

In [19]:
num_steps = 10000
for s in tqdm(range(num_steps)):
    offset = (s*batch_size) % (train_x.shape[0]-batch_size)
    fd = {graph_input:train_x[offset:offset+batch_size,:],
         targets:train_y[offset:offset+batch_size,:]}
    l,_ = sess.run([loss,opt],feed_dict=fd)
    losses.append(l)
    if s % 1000 == 0:
        print('train acc: ',sess.run([train_accuracy],feed_dict=fd))
        print('test acc: ',sess.run([test_accuracy],feed_dict=fd))

  0%|          | 29/10000 [00:00<17:51,  9.30it/s]

('train acc: ', [0.5])
('test acc: ', [0.51999998])


 10%|█         | 1041/10000 [00:03<00:32, 272.40it/s]

('train acc: ', [0.63999999])
('test acc: ', [0.60000002])


 21%|██        | 2051/10000 [00:07<00:29, 272.09it/s]

...

 80%|████████  | 8030/10000 [00:29<00:07, 276.47it/s]

('train acc: ', [0.71999997])
('test acc: ', [0.616])


 91%|█████████ | 9056/10000 [00:33<00:03, 258.31it/s]

('train acc: ', [0.77999997])
('test acc: ', [0.63599998])


100%|██████████| 10000/10000 [00:37<00:00, 269.67it/s]


In [20]:
print('train acc: ',sess.run([train_accuracy],feed_dict=fd))
print('test acc: ',sess.run([test_accuracy],feed_dict=fd))

('train acc: ', [0.82000005])
('test acc: ', [0.63600004])


## Versi 2

In [21]:
w =10
k =10
graph_width = w*k
graph_height = k
nchannels = 18 
batch_size = 50
g = tf.Graph()
with g.as_default():
    graph_input = tf.placeholder(tf.float32,shape=(batch_size,graph_height,graph_width,nchannels))
    targets = tf.placeholder(tf.float32,shape=(batch_size,2))
    pkeep = tf.placeholder(tf.float32)

    testX = tf.constant(test_x,dtype=tf.float32)
    testy = tf.constant(test_y,dtype=tf.float32)
    
    bn_input = createWeightsBN(nchannels)
    W_conv1 = createWeight([k,k,nchannels,2])
    b_conv1 = createBias([2])
    
#     bn_1 = createWeightsBN(16)
#     W_conv2 = createWeight([1,5,16,8])
#     b_conv2 = createBias([8])
    
    bn_2 = createWeightsBN(2)
    W = createWeight([1*(w)*2,2])
    B = createBias([2])
    
    bn_3 = createWeightsBN(2)
    W_fc2 = createWeight([2,2])
    b_fc2 = createBias([2])
    bn_fc2 = createWeightsBN(2)
    
    #model
    def model(data_input):
        data_input_bn = BN(data_input,bn_input)
        Y1 = tf.nn.relu(tf.nn.conv2d(data_input_bn, W_conv1, strides=[1, w, w, 1], padding='VALID') + b_conv1)
        #Y1_drop = tf.nn.dropout(Y1, pkeep)
        
#         y1_bn = BN(Y1,bn_1)
#         Y2 = tf.nn.relu(tf.nn.conv2d(Y1, W_conv2, strides=[1, 1, 1, 1], padding='VALID') + b_conv2)
#         #Y2_drop = tf.nn.dropout(Y2, pkeep)
        
        y1_bn = BN(Y1,bn_2)
        Y1_flat = tf.reshape(Y1, [-1, 1*(w)*2])
        Y_fc1 = tf.nn.relu(tf.matmul(Y1_flat, W) + B)
        Y_drop = tf.nn.dropout(Y_fc1, pkeep)

        Y_conv = tf.matmul(Y_fc1, W_fc2) + b_fc2
        Y_conv_bn = BN(Y_conv,bn_fc2,ri=[0])
        return Y_conv_bn

    #train
    logit_train = model(graph_input)
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits= logit_train,labels = targets))
    
    opt = tf.train.AdamOptimizer(learning_rate=.0001).minimize(loss)
    
    train_accuracy = tf.reduce_mean(tf.cast(tf.equal(
        tf.arg_max(tf.nn.softmax(logit_train),1),tf.arg_max(targets,1)),tf.float32))
    
    #test
    logit_test = model(testX)
    test_accuracy = tf.reduce_mean(tf.cast(tf.equal(
        tf.arg_max(tf.nn.softmax(logit_test),1),tf.arg_max(testy,1)),tf.float32))
    
    init = tf.global_variables_initializer()

In [22]:
sess=tf.Session(graph=g)
sess.run(init)
losses = []

In [23]:
num_steps = 10000
for s in tqdm(range(num_steps)):
    offset = (s*batch_size) % (train_x.shape[0]-batch_size)
    fd = {graph_input:train_x[offset:offset+batch_size,:],
         targets:train_y[offset:offset+batch_size,:]}
    l,_ = sess.run([loss,opt],feed_dict=fd)
    losses.append(l)
    if s % 1000 == 0:
        print('train acc: ',sess.run([train_accuracy],feed_dict=fd))
        print('test acc: ',sess.run([test_accuracy],feed_dict=fd))

  0%|          | 29/10000 [00:00<14:10, 11.72it/s]

('train acc: ', [0.44])
('test acc: ', [0.57599998])


 10%|█         | 1025/10000 [00:04<00:39, 228.48it/s]

('train acc: ', [0.65999997])
('test acc: ', [0.63199997])


 20%|██        | 2034/10000 [00:08<00:28, 282.73it/s]

...

 80%|████████  | 8039/10000 [00:30<00:06, 281.11it/s]

('train acc: ', [0.92000002])
('test acc: ', [0.62])


 90%|█████████ | 9050/10000 [00:33<00:03, 277.18it/s]

('train acc: ', [0.95999998])
('test acc: ', [0.60399997])


100%|██████████| 10000/10000 [00:37<00:00, 269.10it/s]


In [24]:
print('train acc: ',sess.run([train_accuracy],feed_dict=fd))
print('test acc: ',sess.run([test_accuracy],feed_dict=fd))

('train acc: ', [0.94])
('test acc: ', [0.616])
